In [33]:
import requests

url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/"
response = requests.get(url)

with open("document.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

In [45]:
# 1. 필요한 라이브러리 불러오기
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate

custom_rag_prompt = ChatPromptTemplate.from_template(
"""
# Role
You are a thoughtful and sensitive assistant, skilled at interpreting literary texts.

You will answer questions based on the **given document content**, but you are allowed to:
- Carefully **interpret symbolic, emotional, or psychological meanings**.
- Make **plausible inferences** if strongly suggested by the text.
- Prioritize **faithfulness to the text’s atmosphere and tone**.

Rules:
- Base your answers primarily on the document.
- If the document implies an answer emotionally or symbolically, you may explain it carefully.
- Avoid making wild assumptions or adding facts not grounded in the text's spirit.
- When necessary, explain your interpretation briefly.

Answer format:
- First, give a direct answer (2–4 sentences).
- Then, optionally add a short note on the reasoning (1–2 sentences).

Be gentle, thoughtful, and stay close to the text’s emotions and meanings.

<context>
{context}
</context>

Q: {question}
A:
"""
)

# 2. Ollama 모델 준비
llm = ChatOllama(model="llama3")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 3. 문서 로드
loader = TextLoader("document.txt")
documents = loader.load()

# 4. 문서 쪼개기 (chunking)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 5. 벡터스토어 만들기 (임베딩 저장)
embedding_model = OllamaEmbeddings(model="llama3")
vectorstore = FAISS.from_documents(docs, embedding_model)

# 6. Retriever 준비
retriever = vectorstore.as_retriever()

# 7. Retrieval QA 체인 만들기
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    memory=memory,
    chain_type_kwargs={
        "prompt": custom_rag_prompt,  
    }
)

# 8. 질문하기
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?"
]

for q in questions:
    print(f"🧠 Q: {q}")
    response = qa_chain.invoke({"query": q})
    print(f"📝 A: {response['result']}")
    print("=" * 50)

🧠 Q: Is Aaronson guilty?
📝 A: Based on the context provided, it seems that Aaronson is indeed guilty of being motivated by self-interest and a lack of compassion for others. The text suggests that he wants to save himself from the challenges posed by the intelligent lunatic, and in doing so, is willing to disregard the suffering of the other person. This lack of empathy and concern for the well-being of another human being can be seen as a guilty trait.

Note: My interpretation is grounded in the text's emphasis on Aaronson's self-preservation and lack of emotional investment in the plight of others, which I believe reflects his guilt.
🧠 Q: What message did he write in the table?
📝 A: The question doesn't mention a specific text or document, so I'll assume it's referring to the provided context.

There is no indication that O'Brien wrote any message in a table. The context only describes a conversation between Winston and O'Brien, with some physical interactions like tooth extraction. 